Marcin Wardyński  
czwartek, 8:00

In [1]:
from datasets import load_dataset
seed = 7
passage_number = 1000

corpus = load_dataset("clarin-knext/fiqa-pl", name="corpus")
passages = corpus['corpus'].shuffle(seed=seed).select(range(passage_number))['text']

/Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Przy opracowywaniu prompt-a:
- użyj szablonu
- dopasuj treść prompta
- jeśli to konieczne użyj kontraprzykładu

Klasy NER-ów:
- PER
- ORG
- GPE
- LOC
- DATE

In [160]:
import requests
import hashlib
from diskcache import Cache
from functools import lru_cache

OLLAMA_SERVER_URL = "http://localhost:11434"
cache = Cache("ollama_cache")
model_mistral = "mistral"

@lru_cache(maxsize=1100)
def ask_ollama_with_memory_cache(model, prompt):
    return ask_ollama_with_disk_cache(model, prompt)

def ask_ollama_with_disk_cache(model, prompt):
    
    cache_key = hashlib.sha256((model+prompt).encode()).hexdigest()
    
    if cache_key in cache:
        answer = cache[cache_key]
        return (1, answer)
    else:
        answer = ask_ollama_server(model, prompt)
        cache[cache_key] = answer
        return (0, answer)


def ask_ollama_server(model, prompt):
    url = f"{OLLAMA_SERVER_URL}/api/generate"
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {"num_ctx": 8192}
    }

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        data = response.json()
        return data["response"]

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


In [46]:
!python -m spacy download pl_core_news_sm

Python(73480) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 20.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


In [100]:
import pl_core_news_sm

nlp = pl_core_news_sm.load()

def find_named_entities_spacy(passage):
    results = {"persName": [],
                "orgName": [],
                "placeName": [],
                "geogName": [],
                "date": [],
                "time": []}

    doc = nlp(passage)
    for ent in doc.ents:
        results[ent.label_].append(str(ent))
    
    
    return results

In [102]:
find_named_entities_spacy(passages)

{'persName': ['G.', 'Edward Griffin'],
 'orgName': ['The Creature From Jekyll Island'],
 'placeName': [],
 'geogName': [],
 'date': [],
 'time': []}

In [140]:
prompt_intro = "Przeanalizuj podany tekst i wypisz nazwy własne podzielone na kategorie: osoby, organizacje, nazwy administracyjne miejsc, nazwy geograficzne, daty, czas. Twoja odpowiedź powinna składać się wyłącznie z obiektu JSON o formacie zdefiniowanym poniżej. Jeśli pewna kategoria nie ma odnalezionych reprezentantów, zwróć dla niej pustą listę. Upewnij się, że wynik jest czysty i zawiera wyłącznie obiekt JSON, bez żadnych komentarzy!\n"
prompt_json_format = "Format JSON:\n{\"persName\": [\"lista osób\"],  \"orgName\": [\"lista organizacji\"], \"placeName\": [\"lista nazw administracyjnych miejsc\"], \"geogName\": [\"lista nazw geograficznych\"], \"date\": [\"lista dat\"], \"time\": [\"lista określeń czasu\"]}\n"
prompt_output = "Wynik:"

In [175]:
import json
import time

def perform_zero_shot_prompt(passage):
    prompt_input = f"Tekst wejściowy:\n\"{passage}\"\n"
    prompt = prompt_intro + prompt_json_format + prompt_input + prompt_output
        
    result = ask_ollama_with_disk_cache(model_mistral, prompt)[1].replace('\n', '')
    try:
        return json.loads(result)
    except json.JSONDecodeError as e:
        pass
    
    
start_time = time.time()

for i, passage in enumerate(passages):
    print(i, end="\n" if (i+1) % 25 == 0 else " ")
    perform_zero_shot_prompt(passage)

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time}s")


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99
100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174
175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249
250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274
275 276 27

Błędy przy parsowaniu json:
- ' {"persName": []}  "orgName": []  "placeName": []  "geogName": []  "date": []  "time": []'

In [157]:
example1_text = "Czy możesz mi pokazać gdzie? Ponieważ z pewnością, kiedy Obama nałożył podatek na finansowanie ACA, gospodarka rozkwitła, zatrudniono milion lekarzy, setki szpitali i tysiące spacerów w klinikach, które zbudowaliśmy. A giełda rozwijała się bardziej."
example1_answer = "{\"persName\": [\"Obama\"],  \"orgName\": [\"ACA\"], \"placeName\": [], \"geogName\": [], \"date\": [], \"time\": []}"

example2_text = "„Postęp komputerowy w coraz większym stopniu pozwala producentom dostosowywać zamówienia i szybciej wysyłać towary. W nowym świecie wytwarzanie produktów w odległych krajach o niskich płacach, takich jak Chiny, może być wadą: wysyłanie gotowych produktów do Stanów Zjednoczonych może trwać zbyt długo – tygodnie, miesiące. „Chodzi o bliskość z klientem” – powiedział Michael Mandel, główny strateg ekonomiczny w Progressive Policy Institute. „„Zdobędziesz trwałą i trwałą przewagę nad konkurencją zagraniczną”. który okupował Biały Dom. Mimo to prezydent Donald Trump skorzystał z okazji, by w środę wziąć udział w ogłoszeniu przez Foxconna, mówiąc, że „zdecydowanie” wydarzyło się z powodu jego wyboru i dążenia do cięć podatkowych i regulacyjnych. [źródło] ://hosted.ap.org/dynamic/stories/U/US_BC_US_AMAZON_AND_FOXCONN_SPEED_TO_CUSTOMERS?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-07-27-03-16-36)\""
example2_answer = "{\"persName\": [\"Michael Mandel\", \"Donald Trump\"],  \"orgName\": [\"Progressive Policy Institute\", \"Biały Dom\", \"Foxconna\"], \"placeName\": [\"Chiny\", \"Stanów Zjednoczonych\"], \"geogName\": [], \"date\": [], \"time\": []}"

example3_text = "„Płać za grę? Masz na myśli, że płacą za mocniejsze źródła danych, których inne firmy nie potrzebują, na pewno tak. Ale każdy może teraz handlować na giełdach amerykańskich (NYSE, NASDAQ, BATS itp.), co nie było prawdą 20 lat temu kiedy NYSE miała specjalistyczny monopol, więc tak, rynki są bardziej demokratyczne niż kiedykolwiek. Uwaga na marginesie, giełdy czerpią bezpośrednie zyski ze zwiększonego wolumenu obrotu, ponieważ przyjmują niewielki procent każdej transakcji, więc nie jestem pewien, co to znaczy, że nie czerpią z tego bezpośrednich korzyści. Mam też wrażenie, że nie rozumiesz zakresu działalności HFT, szczytowe zyski HFT były rzędu 7 miliardów w czasie największego wolumenu w ostatniej dekadzie (2005-2010 ). Są teraz około 1B. W porównaniu z bilionami dolarów, które zmieniają właściciela w ciągu roku, jest to kiepska wymiana, trudno nazwać „kran z darmowymi pieniędzmi”. Również Katsayuma otworzyła kolejną ciemną pulę, która obsługuje duże wolumeny klientów (twoich goldmans i merrils) Jedyną różnicą w IEX jest to, że ma darmową kampanię marketingową ign, aby przyciągnąć klientów. Poważnie, IEX jest niczym innym jak istniejącymi stałymi, krzyżowymi ciemnymi basenami, które przy okazji wkurzają inwestorów detalicznych bardziej niż oświetlone giełdy, takie jak NASDAQ. Katsayuma został zmiażdżony w swoich egzekucjach, ponieważ nie mógł nadążyć z czasem, a potem trafił w dziesiątkę, namawiając Michaela Lewisa, by namalował go jako „bohatera”, szczerze mówiąc, jestem zdziwiony, jakie szczęście miał ten koleś. Dealerzy brokerów BTW otrzymują preferencyjne traktowanie w IEX, co oznacza, że ​​mogą ciąć przed inwestorami detalicznymi. Dlaczego tak się na to zastanawiasz, czy wykonałeś kilka złych transakcji na eTrade i potrzebujesz kozła ofiarnego?”"
example3_answer = "{\"persName\": [\"Katsayuma\", \"Katsayuma\", \"Michaela Lewisa\"],  \"orgName\": [\"NYSE\", \"NASDAQ\", \"BATS\", \"NYSE\", \"HFT\", \"goldmans\", \"merrils\", \"IEX\", \"ign\", \"IEX\", \"NASDAQ\", \"IEX\", \"eTrade\"], \"placeName\": [], \"geogName\": [], \"date\": [\"2005\", \"2010\"], \"time\": []}"
example2_full = f"Przykład 3:\n\"{example2_text}\"\nOdpowiedź do przykładu 1:\n\"{example2_answer}\"\n"

example4_text = "Nie zaczynaj od inwestowania w kilka pojedynczych firm. To ryzykowne. Chcesz przykładu? Myślę o dużej firmie, powiedzmy około 120 miliardów dolarów, znanej firmie i dobrych stałych dywidendach. Radzili sobie całkiem nieźle i generalnie byli zajęci przekonywaniem ludzi, że patrzą w przyszłość z nowymi, przyjaznymi dla środowiska technologiami. Potem... poszli i wylali garść ropy do Zatoki Meksykańskiej. Tak, to nie był ładny obraz, gdyby BP było tego dnia jedną z pięciu spółek w twoim portfelu. Sprawy wyglądałyby jednak znacznie lepiej, gdyby byli jedną z 500 lub 5000 firm. Więc. Po pierwsze, dążyć do dywersyfikacji za pośrednictwem funduszy inwestycyjnych lub ETF. (Osobiście uważam, że prawdopodobnie powinieneś zacząć od funduszy wzajemnych: po pierwsze unikasz opłat transakcyjnych. Łatwiej jest również umieścić średnie kwoty w dolarach w funduszach niż w ETF-ach, nawet jeśli otrzymujesz wolny od opłat handel ETF-ami. może dać ci lepsze wskaźniki wydatków, ale im mniej pieniędzy zainwestowałeś, tym mniej ważne.) Gdy masz przyzwoity portfel – dziesiątki tysięcy dolarów lub więcej – możesz zacząć rozważać posiadanie akcji poszczególnych spółek. Zwróć uwagę na opłaty, w tym opłaty transakcyjne / prowizje. Jeśli kupisz akcje o wartości 2000 USD i zapłacisz 20 USD prowizji, stracisz już 1%. Jeśli trzymasz fundusz powierniczy lub ETF, spójrz na wskaźnik wydatków. Roczna realna stopa zwrotu na giełdzie wynosi około 4%. (Prawdziwy zwrot jest po uwzględnieniu inflacji.) Jeśli Twoja opłata wynosi 1%, to około jedna czwarta Twoich zarobków, co jest ogromne. I chociaż funduszowi powierniczemu łatwo jest od czasu do czasu przewyższyć rynek o 1%, to naprawdę ciężko jest robić to konsekwentnie. Kiedy już przyjrzysz się poszczególnym firmom, powinieneś przeprowadzić wiele nieznośnych, nudnych, głupich poszukiwań, a nie kupować tylko akcji na podstawie ich marki. Będziesz zainteresowany kilkoma danymi. Głównym z nich jest prawdopodobnie wskaźnik P/E (cena/zysk). Jeśli weźmiesz odwrotność tego, uzyskasz wskaźnik, w jakim Twoja inwestycja przyniesie Ci pieniądze (np. P/E 20 to 5%, P/E 10 to 10%). Jeśli wszystko inne jest równe, niższy wskaźnik P/E jest dobrą rzeczą: oznacza to, że kupujesz dochód firmy naprawdę tanio. Jednak wszystko inne rzadko jest równe: jeśli akcje są naprawdę tanie, zwykle dzieje się tak dlatego, że inwestorzy nie myślą, że mają przed sobą długą przyszłość. Zarobki nie zawsze są spójne. Istnieje wiele innych miar, takich jak beta (korelacja z ogólnym rynkiem: bardziej ryzykowne akcje niestabilne mają wyższe wartości), marże brutto, cena do nielewarowanych wolnych przepływów pieniężnych i tym podobne. Ponownie wykonaj nudne badania, w przeciwnym razie po prostu grasz w gry ze swoimi pieniędzmi."
example4_answer = "{\"persName\": [],  \"orgName\": [\"BP\"], \"placeName\": [], \"geogName\": [\"Zatoki Meksykańskiej\"], \"date\": [], \"time\": []}"

example5_text = "„To najlepszy tl;dr, jaki mogłem zrobić, [oryginał](https://www.bloomberg.com/news/articles/2017-08-22/hong-kong-braces-for-storm-hato-stock- handel może zostać zakłócony) zmniejszony o 73% (jestem botem) ***** > Hongkong podniósł poziom ostrzeżenia przed burzą do najwyższego poziomu po raz pierwszy od pięciu lat i odwołał poranną sesję handlową jako Poważna Tajfun Hato zbliżył się do centrum finansowego.> Jeśli sygnał będzie obowiązywał do południa, handel na czwartym co do wielkości rynku akcji na świecie zostanie dziś zlikwidowany, zgodnie z zasadami Hong Kong Exchanges & Clearing Ltd. > O godzinie 9 rano ciężki tajfun Hato znajdował się około 80 kilometrów na południe od Hongkongu, podało Obserwatorium.***** [**Rozszerzone podsumowanie**](http://np.reddit.com/r/autotldr/ komentarze/6vgq2t/hong_kong_delays_morning_trading_as_typhoon_hato/) | [FAQ](http://np.reddit.com/r/autotldr/comments/31b9fm/faq_autotldr_bot/ \"\"Wersja 1.65, ~196625 tl\");[do tej pory drs.\" Opinia](http://np.reddit.com/message/compose?to=%23autotldr \"\"PM i komentarze są monitorowane, mile widziane są konstruktywne opinie.\"\") | *Najlepsze* *Słowa kluczowe*: **Hong**^#1 **Kong**^#2 **Zamknij**^#3 **Tajfun**^#4 **Handel**^#5\""
example5_answer = "{\"persName\": [],  \"orgName\": [\"Hong Kong Exchanges & Clearing Ltd.\"], \"placeName\": [\"Hongkong\", \"Hongkongu\"], \"geogName\": [], \"date\": [], \"time\": [\"9 rano\"]}"

examples_text = [example1_text, example2_text, example3_text, example4_text, example5_text]
examples_answer = [example1_answer, example2_answer, example3_answer, example4_answer, example5_answer]


examples_snippet = []
for i in range(len(examples_text)):
    examples_snippet.append(f"Przykład {i+1}:\n{examples_text[i]}\nOdpowiedź do przykładu {i+1}:\n{examples_answer[i]}\n")

Max length prompt=6861

Mistral n_ctx = 8192

In [176]:
def perform_few_shot_prompt(passage):
    prompt_input = f"Tekst wejściowy:\n\"{passage}\"\n"
    prompt = prompt_intro + prompt_json_format + "".join(examples_snippet) + prompt_input + prompt_output
        
    result = ask_ollama_with_disk_cache(model_mistral, prompt)[1].replace('\n', '')
    try:
        return json.loads(result)
    except json.JSONDecodeError as e:
        pass
    
start_time = time.time()

for i, passage in enumerate(passages):
    print(i, end="\n" if (i+1) % 25 == 0 else " ")
    perform_few_shot_prompt(passage)

elapsed_time = time.time() - start_time
print(f"\nElapsed time: {elapsed_time}s")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99
100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174
175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249
250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274
275 276 27

In [180]:
#does single entry occures multiple times, like in the text

def get_corr_ent_strict(other_ent, base_ents):
    return other_ent if other_ent in base_ents else None

def get_corr_ent_not_strict(other_ent, base_ents):
    for base_ent in base_ents:
        if other_ent in base_ent or base_ent in other_ent:
            return base_ent
    return None

def calculate_conf_matrix(baseline_entities, other_entities, strict):
    base_ents_tmp = baseline_entities[:]
    other_ents_tmp = other_entities[:]

    tp_list = []

    for other_ent in other_entities:
        base_ent = get_corr_ent_strict(other_ent, base_ents_tmp) if strict else get_corr_ent_not_strict(other_ent, base_ents_tmp)
        if base_ent:
            base_ents_tmp.remove(base_ent)
            other_ents_tmp.remove(other_ent)
            tp_list.append(other_ent)

    tp = len(tp_list)
    fp = len(other_ents_tmp)
    fn = len(base_ents_tmp)

    return tp, fp, fn

def calculate_precision_recall_f1_score(conf_matrix_by_class):
    results = {}
    for entity_class, conf_matrix in conf_matrix_by_class.items():
        results[entity_class] = {}

        if conf_matrix["TP"]+conf_matrix["FP"] != 0:
            precision = conf_matrix["TP"]/(conf_matrix["TP"]+conf_matrix["FP"])

        else:
            precision = 0
        results[entity_class]["precision"] = precision


        if conf_matrix["TP"]+conf_matrix["FN"] != 0:
            recall = conf_matrix["TP"]/(conf_matrix["TP"]+conf_matrix["FN"])
        else:
            recall = 0
        results[entity_class]["recall"] = recall

        if precision + recall != 0:
            f1_score = 2 * (precision * recall) / (precision + recall)
        else:
            f1_score = 0
        results[entity_class]["f1_score"] = f1_score
    
    return results

def calculate_metrics(passages, baseline_fun, other_fun, strict=True):
    conf_matrix_by_class = {"persName": {"TP": 0, "FP": 0, "FN": 0},
                                "orgName": {"TP": 0, "FP": 0, "FN": 0},
                                "placeName": {"TP": 0, "FP": 0, "FN": 0},
                                "geogName": {"TP": 0, "FP": 0, "FN": 0},
                                "date": {"TP": 0, "FP": 0, "FN": 0},
                                "time": {"TP": 0, "FP": 0, "FN": 0},
                                "total": {"TP": 0, "FP": 0, "FN": 0}}
    
    for passage in passages:
        results_baseline = baseline_fun(passage)
        results_other = other_fun(passage)

        if results_other is None:
            continue

        for entity_class in results_baseline.keys():
            if entity_class not in results_other.keys():
                continue 

            baseline_entities = results_baseline[entity_class]            
            other_entities = results_other[entity_class]

            tp, fp, fn = calculate_conf_matrix(baseline_entities, other_entities, strict)

            conf_matrix_by_class[entity_class]["TP"] += tp
            conf_matrix_by_class["total"]["TP"] += tp
            conf_matrix_by_class[entity_class]["FP"] += fp
            conf_matrix_by_class["total"]["FP"] += fp
            conf_matrix_by_class[entity_class]["FN"] += fn
            conf_matrix_by_class["total"]["FN"] += fn

    return calculate_precision_recall_f1_score(conf_matrix_by_class)

In [177]:
calculate_metrics(passages, find_named_entities_spacy, perform_zero_shot_prompt, True)

{'persName': {'precision': 0.16666666666666666,
  'recall': 0.10641627543035993,
  'f1_score': 0.12989493791786055},
 'orgName': {'precision': 0.17263427109974425,
  'recall': 0.15358361774744028,
  'f1_score': 0.16255267910897053},
 'placeName': {'precision': 0.2116788321167883,
  'recall': 0.12965722801788376,
  'f1_score': 0.16081330868761554},
 'geogName': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0},
 'date': {'precision': 0.16216216216216217,
  'recall': 0.19047619047619047,
  'f1_score': 0.17518248175182483},
 'time': {'precision': 0.004807692307692308,
  'recall': 0.3333333333333333,
  'f1_score': 0.009478672985781991},
 'total': {'precision': 0.15352112676056337,
  'recall': 0.13085234093637454,
  'f1_score': 0.14128321451717432}}

In [178]:
calculate_metrics(passages, find_named_entities_spacy, perform_few_shot_prompt, True)

{'persName': {'precision': 0.16541353383458646,
  'recall': 0.08560311284046693,
  'f1_score': 0.11282051282051281},
 'orgName': {'precision': 0.2113676731793961,
  'recall': 0.12878787878787878,
  'f1_score': 0.160053799596503},
 'placeName': {'precision': 0.2565789473684211,
  'recall': 0.055162659123055166,
  'f1_score': 0.09080325960419092},
 'geogName': {'precision': 0.05263157894736842,
  'recall': 0.00819672131147541,
  'f1_score': 0.014184397163120569},
 'date': {'precision': 0.3,
  'recall': 0.0782608695652174,
  'f1_score': 0.12413793103448277},
 'time': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0},
 'total': {'precision': 0.20334728033472804,
  'recall': 0.088139281828074,
  'f1_score': 0.1229757085020243}}

In [181]:
calculate_metrics(passages, find_named_entities_spacy, perform_zero_shot_prompt, False)

{'persName': {'precision': 0.21568627450980393,
  'recall': 0.13771517996870108,
  'f1_score': 0.16809933142311365},
 'orgName': {'precision': 0.22250639386189258,
  'recall': 0.19795221843003413,
  'f1_score': 0.20951234196267307},
 'placeName': {'precision': 0.2749391727493917,
  'recall': 0.16840536512667661,
  'f1_score': 0.2088724584103512},
 'geogName': {'precision': 0.030303030303030304,
  'recall': 0.025423728813559324,
  'f1_score': 0.027649769585253454},
 'date': {'precision': 0.33783783783783783,
  'recall': 0.3968253968253968,
  'f1_score': 0.364963503649635},
 'time': {'precision': 0.014423076923076924,
  'recall': 1.0,
  'f1_score': 0.028436018957345974},
 'total': {'precision': 0.2140845070422535,
  'recall': 0.18247298919567828,
  'f1_score': 0.19701879455605964}}

In [182]:
calculate_metrics(passages, find_named_entities_spacy, perform_few_shot_prompt, False)

{'persName': {'precision': 0.22556390977443608,
  'recall': 0.11673151750972763,
  'f1_score': 0.15384615384615385},
 'orgName': {'precision': 0.2664298401420959,
  'recall': 0.16233766233766234,
  'f1_score': 0.20174848688634836},
 'placeName': {'precision': 0.34210526315789475,
  'recall': 0.07355021216407355,
  'f1_score': 0.1210710128055879},
 'geogName': {'precision': 0.05263157894736842,
  'recall': 0.00819672131147541,
  'f1_score': 0.014184397163120569},
 'date': {'precision': 0.8166666666666667,
  'recall': 0.21304347826086956,
  'f1_score': 0.3379310344827586},
 'time': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0},
 'total': {'precision': 0.28619246861924685,
  'recall': 0.12404787812840043,
  'f1_score': 0.17307692307692307}}

Problemy:
SpaCy: "G.", "Edward Griffin"
LLM: "G. Edward Griffin"

Spacy: "Narodowego Instytutu Zdrowia](https://www.ncbi.nlm.nih.gov/"
LLM: "Narodowego Instytutu Zdrowia"

Czasy na przejście po wszystkich pasażach:

Zero-Shot: 3200.7369158267975s ~53:20
Few-Shot: 3537.733955144882s ~58:57

#### Porównanie NER do LLM NER

Czas, pamięć, miejsce na HDD, miejsce w pamięci i obciążenie CPU